In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
import malaya_speech.train as train
import malaya_speech
import malaya_speech.train.model.marblenet as marblenet
import tensorflow as tf
import malaya_speech.config
import numpy as np


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
config = malaya_speech.config.ctc_featurizer_config
config['feature_type'] = 'mfcc'
featurizer = malaya_speech.tf_featurization.STTFeaturizer(**config)
n_mels = featurizer.num_feature_bins

In [4]:
def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    s = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(s)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = s
    example['inputs_length'] = length

    return example


def parse(serialized_example):

    data_fields = {
        'waveforms': tf.VarLenFeature(tf.float32),
        'targets': tf.VarLenFeature(tf.int64),
    }
    features = tf.parse_single_example(
        serialized_example, features=data_fields
    )
    for k in features.keys():
        features[k] = features[k].values

    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['inputs', 'inputs_length', 'targets']:
            features.pop(k, None)

    return features


def get_dataset(files, batch_size=32, shuffle_size=32, num_cpu_threads=6,
                thread_count=24, is_training=True):
    def get():
        if is_training:
            d = tf.data.Dataset.from_tensor_slices(tf.constant(files))
            d = d.repeat()
            d = d.shuffle(buffer_size=len(files))
            cycle_length = min(num_cpu_threads, len(files))
            d = d.interleave(
                tf.data.TFRecordDataset,
                cycle_length=cycle_length,
                block_length=batch_size)
            d = d.shuffle(buffer_size=100)
        else:
            d = tf.data.TFRecordDataset(files)
            d = d.repeat()
        d = d.map(parse, num_parallel_calls=thread_count)
        d = d.padded_batch(
            batch_size,
            padded_shapes={
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
            },
            padding_values={
                'inputs': tf.constant(0, dtype=tf.float32),
                'inputs_length': tf.constant(0, dtype=tf.int32),
                'targets': tf.constant(0, dtype=tf.int64),
            },
        )
        return d

    return get

In [5]:
dev_files = tf.io.gfile.glob(
    'vad/data/vad-dev*'
)
dev_dataset = get_dataset(dev_files, is_training=False)()
features = dev_dataset.make_one_shot_iterator().get_next()
features



Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'targets': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int64>,
 'inputs': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?, 80) dtype=float32>,
 'inputs_length': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?) dtype=int32>}

In [6]:
Y = tf.cast(features['targets'][:, 0], tf.int32)
model = marblenet.Model(features['inputs'], features['inputs_length'][:, 0], factor=3, training=True)
logits = tf.reduce_sum(model.logits['outputs'], axis=1)
logits = tf.layers.dense(logits, 2)

loss = tf.reduce_mean(
    tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=Y
    )
)

accuracy = tf.metrics.accuracy(
    labels=Y, predictions=tf.argmax(logits, axis=1)
)


Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Use keras.layers.Dense instead.


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'marblenet-factor3/model.ckpt-2000000')

INFO:tensorflow:Restoring parameters from marblenet-factor3/model.ckpt-2000000


In [9]:
from tqdm import tqdm

eval_step = 10000
rs = []
for i in tqdm(range(eval_step)):
    r = sess.run([Y, logits])
    rs.append((r[0] == r[1].argmax(axis = 1)).mean())

100%|██████████| 10000/10000 [00:57<00:00, 174.92it/s]


In [10]:
np.mean(rs)

0.83855625